#                                                                    Nintendo Switch API


## Libraries


In [1]:
!pip install requests beautifulsoup4 fastapi uvicorn


In [66]:
!pip install selenium

   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.5 MB 1.5 MB/s eta 0:00:06
   --- ------------------------------------ 0.8/9.5 MB 1.5 MB/s eta 0:00:07
   ------ --------------------------------- 1.6/9.5 MB 2.2 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.5 MB 2.3 MB/s eta 0:00:04
   ------------- -------------------------- 3.1/9.5 MB 2.8 MB/s eta 0:00:03
   ---------------- ----------------------- 3.9/9.5 MB 3.1 MB/s eta 0:00:02
   ---------------------- ----------------- 5.2/9.5 MB 3.5 MB/s eta 0:00:02
   --------------------------- ------------ 6.6/9.5 MB 3.8 MB/s eta 0:00:01
   --------------------------------- ------ 7.9/9.5 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.5 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: at

In [71]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importamos las librerías necesarias

from bs4 import BeautifulSoup
import requests


## Code

In [7]:
page = requests.get(url)

In [9]:
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
print(soup.prettify())

## Código para el web Scraping

In [6]:
#Importamos librerías
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

#Definimos la función
def scrape_nintendo_data_with_selenium(url_entry):
    # Configuración de Selenium
    options = Options()
    options.add_argument("--headless")  # Ejecutamos en modo headless
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Navegamos por la página
    driver.get(url_entry)

    # Scroll dinámico para cargar contenido
    for _ in range(3):  # Hacer scroll varias veces
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        time.sleep(2)

    # Esperamos a que los bloques de juegos estén presentes
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "searchresult_row")))

    # Encontramos todos los bloques de juegos
    game_blocks = driver.find_elements(By.CLASS_NAME, "searchresult_row")
    print(f"Bloques encontrados: {len(game_blocks)}")  # Depuración

    #Listas a completar
    titles = []
    prices = []
    categories = []
    release_dates = []
    descriptions = []  

    for idx, block in enumerate(game_blocks):
        print(f"Procesando bloque {idx + 1}")  # Depuración

        # Extraemos título
        try:
            title_element = block.find_element(By.CLASS_NAME, "page-title-text")
            title = title_element.text.strip()
        except Exception:
            title = "Título no disponible"

        print(f"Título: {title}")  # Depuración

        # Extraemos precio
        try:
            price_element = block.find_element(By.CLASS_NAME, "price-small")
            price = price_element.text.strip()
        except Exception:
            price = "Precio no disponible"

        print(f"Precio: {price}")  # Depuración

        # Extraemos categorías
        try:
            category_element = block.find_element(By.CLASS_NAME, "hidden-xs")
            category = category_element.text.strip()
        except Exception:
            category = "Categoría no disponible"

        print(f"Categoría: {category}")  # Depuración

        # Extraemos fecha de lanzamiento
        try:
            release_date_element = block.find_element(By.CLASS_NAME, "page-data")
            release_date_text = release_date_element.text.strip()
            print(f"Texto completo de la fecha de lanzamiento: {release_date_text}")  # Depuración

            # Usamos expresión regular para encontrar fechas
            date_match = re.search(r'(\d{2}-\d{2}-\d{4}|\d{4})', release_date_text)
            if date_match:
                release_date = date_match.group(0)
            else:
                release_date = "Fecha no disponible"
        except Exception:
            release_date = "Fecha no disponible"

        print(f"Fecha de lanzamiento: {release_date}")  # Depuración

        # Extraemos descripción
        try:
            description_element = block.find_element(By.CLASS_NAME, "hidden-xs.visible-sm.visible-md.visible-lg")
            description = description_element.text.strip()
        except Exception:
            description = "Descripción no disponible"

        print(f"Descripción: {description}")  # Depuración

        # Agregamos datos a las listas
        titles.append(title)
        prices.append(price)
        categories.append(category)
        release_dates.append(release_date)
        descriptions.append(description)  # Agregar la descripción

    # Cerramos el navegador
    driver.quit()

    return titles, prices, categories, release_dates, descriptions




## Web Scraping por página

In [72]:
# Variables y listas a poblar.
url = "https://www.nintendo.com/es-es/Buscar/Buscar-299117.html?f=147394-5-81"
titles = []
categories = []
prices = []
release_dates = []
descriptions = []  

# Se recomienda el procesamiento de bloques de 50 páginas para evitar bloqueos de la página.
for i in range(1, 587):  
    time.sleep(3)
    if i == 1:
        page_titles, page_prices, page_categories, page_release_dates, page_descriptions = scrape_nintendo_data_with_selenium(url)
    else:
        page_titles, page_prices, page_categories, page_release_dates, page_descriptions = scrape_nintendo_data_with_selenium(url + "&p=" + str(i))

    # Agregamos los datos a las listas generales
    titles.extend(page_titles)
    prices.extend(page_prices)
    categories.extend(page_categories)
    release_dates.extend(page_release_dates)
    descriptions.extend(page_descriptions)

# Ahora obtenemos las listas `titles`, `prices`, `categories`, `release_dates` y `descriptions`
print("Títulos:", titles)
print("Precios:", prices)
print("Categorías:", categories)
print("Fechas de lanzamiento:", release_dates)
print("Descripciones:", descriptions)



Bloques encontrados: 27
Procesando bloque 1
Título: Título no disponible
Precio: Precio no disponible
Categoría: Categoría no disponible
Fecha de lanzamiento: Fecha no disponible
Descripción: Descripción no disponible
Procesando bloque 2
Título: 
Precio: 
Categoría: 
Texto completo de la fecha de lanzamiento: 
Fecha de lanzamiento: Fecha no disponible
Descripción: 
Procesando bloque 3
Título: Título no disponible
Precio: Precio no disponible
Categoría: Categoría no disponible
Fecha de lanzamiento: Fecha no disponible
Descripción: Descripción no disponible
Procesando bloque 4
Título: Guns, Gore and Cannoli 2
Precio: 12,99 €*
Categoría: • Acción, Arcade, Plataformas
Texto completo de la fecha de lanzamiento: Nintendo Switch • 02-08-2018 • Acción, Arcade, Plataformas
Fecha de lanzamiento: 02-08-2018
Descripción: La historia de Vinnie Cannoli continúa y, en esta ocasión, se trata de algo personal.
Procesando bloque 5
Título: Iconoclasts
Precio: 19,99 €4,99 €*
Categoría: • Acción, Aventura


## Generación de archivos CSV

In [74]:
with open("nintendo_data.csv", mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(["Título", "Precio", "Categoría", "Fecha de lanzamiento", "Descripción"])  # Encabezados
    for title, price, category, release_date, description in zip(titles, prices, categories, release_dates, descriptions):
        writer.writerow([title, price, category, release_date, description])


## Concatenación de los dataframe

In [2]:
import os
import pandas as pd

# Ruta donde están los archivos CSV
folder_path = r"C:\Users\Usuario\Desktop\varios\CESAR\master\projecto nintendo"
output_file = os.path.join(folder_path, "nintendo_master.csv")

# Obtenemos lista de archivos CSV en la carpeta
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Lista para almacenar los DataFrames
dfs = []

# Leer y concatenar los CSV
for i, file in enumerate(csv_files):
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, encoding="utf-8-sig")
    
    # Solo mantenemos los encabezados del primer archivo, evitamos duplicado de los mismos.
    if i == 0:
        dfs.append(df)
    else:
        dfs.append(df.iloc[1:])  # Omitir la primera fila (encabezados) en archivos siguientes

# Concatenamos todos los DataFrames
df_master = pd.concat(dfs, ignore_index=True)

# Guardamos el archivo unificado
df_master.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"Unificación completa. Archivo guardado en: {output_file}")
print(f"Total de líneas: {len(df_master)}")



Unificación completa. Archivo guardado en: C:\Users\Usuario\Desktop\varios\CESAR\master\projecto nintendo\nintendo_master.csv
Total de líneas: 15805


## Limpieza de los datos

In [8]:
import pandas as pd

# Ruta del archivo original
file_path = r"C:\Users\Usuario\Desktop\varios\CESAR\master\projecto nintendo\nintendo_master.csv"
output_path = r"C:\Users\Usuario\Desktop\varios\CESAR\master\projecto nintendo\nintendo_master_limpio.csv"

# Cargar el CSV
df = pd.read_csv(file_path, encoding="utf-8-sig")

# Eliminar filas donde la primera columna ("Título") esté vacía o contenga valores no deseados
df = df[~df["Título"].isin(["Título", "Título no disponible"])]  # Elimina filas con esos valores
df = df[df["Título"].notna()]  # Elimina filas donde la celda está vacía (NaN)

# Guardar el archivo limpio
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"Limpieza completa. Archivo guardado en: {output_path}")
print(f"Total de líneas después de la limpieza: {len(df)}")


Limpieza completa. Archivo guardado en: C:\Users\Usuario\Desktop\varios\CESAR\master\projecto nintendo\nintendo_master_limpio.csv
Total de líneas después de la limpieza: 14058
